In [ ]:
import numpy as np
import itertools
import PIL.Image
import matplotlib.pyplot as plt
from pathlib import Path
from dataclasses import dataclass
# plt.rcParams['figure.figsize'] = [25, 15]


In [ ]:
def first_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    return np.min( np.where(mask.any(axis=axis), mask.argmax(axis=axis), 100000) )
def last_nonzero(arr, axis):
    flipped_nz = first_nonzero(np.flip(arr), axis)
    return arr.shape[axis] - flipped_nz

In [ ]:
@dataclass
class Features:
    weight: list
    weight_norm: list
    centroid: tuple
    centroid_norm: tuple
    axes_momentum: tuple
    axes_momentum_norm: tuple
    profiles: list

class FeaturesCalculator:
    @staticmethod
    def split_to_quadrants(img):
        sz = img.shape
        hz = ( img.shape[0] // 2, img.shape[1] // 2 )
        return [
            img[0:hz[0], 0:hz[1]],
            img[0:hz[0], hz[1]:sz[1]],
            img[hz[0]:sz[0], 0:hz[1]],
            img[hz[0]:sz[0], hz[1]:sz[1]]
        ]

    @staticmethod
    def weight_per_quadrant(img):
        return list(map( lambda q: np.sum(q), FeaturesCalculator.split_to_quadrants( img ) ))

    @staticmethod
    def weight_norm_per_quadrant(img):
        return list(map( lambda q: np.sum(q) / ( q.shape[0] * q.shape[1] ), FeaturesCalculator.split_to_quadrants( img ) ))

    @staticmethod
    def axes_reduce(img, fs):
        if not isinstance(fs, tuple):
            fs = (fs, fs)

        meanx = 0
        meany = 0
        for y, x in itertools.product( range( img.shape[0] ), range( img.shape[1] ) ):
            meany += fs[0]( y ) * img[y, x]
            meanx += fs[1]( x ) * img[y, x]

        s = np.sum(img)
        return meany/s, meanx/s

    @staticmethod
    def centroid(img):
        return FeaturesCalculator.axes_reduce(img, lambda x: x)

    @staticmethod
    def centroid_norm(img):
        c = FeaturesCalculator.centroid( img )
        return c[0] / img.shape[0], c[1] / img.shape[1]

    @staticmethod
    def axes_momentum(img):
        c = FeaturesCalculator.centroid(img)
        return FeaturesCalculator.axes_reduce(img, ( lambda y: (y - c[0])**2, lambda x: (x - c[1])**2 ) )

    @staticmethod
    def axes_momentum_norm(img):
        m = FeaturesCalculator.axes_momentum(img)
        n = img.shape[0] ** 2 + img.shape[1] ** 2
        return m[0] / n, m[1] / n

    @staticmethod
    def proj(img, axis):
        return np.sum(img, axis)

    def __init__(self, img):
        self.features = Features(
            weight = FeaturesCalculator.weight_per_quadrant( img ),
            weight_norm = FeaturesCalculator.weight_norm_per_quadrant( img ),
            centroid = FeaturesCalculator.centroid( img ),
            centroid_norm = FeaturesCalculator.centroid_norm( img ),
            axes_momentum = FeaturesCalculator.axes_momentum( img ),
            axes_momentum_norm = FeaturesCalculator.axes_momentum_norm( img ),
            profiles=[FeaturesCalculator.proj( img, 0 ), FeaturesCalculator.proj( img, 1 )]
        )

alphabet_path = '/home/andrew/Projects/MEPhI/ImageAndAudioProcessing/data/glag_letters/'
alphabet = []
for i in range(41):
    letter_path = alphabet_path + f'{i}.tif'
    letter = np.array( PIL.Image.open( letter_path ) ).astype(float)
    alphabet.append((FeaturesCalculator(letter).features, letter))

In [ ]:
def flatten_features(f: Features):
    return [item for lst in [f.centroid_norm, f.weight_norm, f.axes_momentum_norm] for item in lst]

def dist(f1: Features, f2: Features):
    l1 = np.array(flatten_features(f1))
    l2 = np.array(flatten_features(f2))

    return np.linalg.norm(l1 - l2, 2)

In [ ]:
base_path = '/home/andrew/Projects/MEPhI/ImageAndAudioProcessing/data/segmented_letters/'
for i in range(39):
    letter_path = Path(base_path + f'{i}.tif')
    img = np.array(PIL.Image.open(letter_path)).astype(float)
    left = first_nonzero(img, 1)
    right = last_nonzero(img, 1)
    top = first_nonzero(img, 0)
    bottom = last_nonzero(img, 0)

    img = img[top:bottom, left:right]

    img_feat = FeaturesCalculator(img).features
    probs = list(map( lambda t: dist(img_feat, t), map( lambda t: t[0], alphabet ) ))
    rs = np.argsort(probs)
    r = rs[0]
    # print(letter_path.name, r)

    # if i < 25:
    plt.figure()
    plt.subplot(1, 4, 1)
    plt.title('image')
    plt.imshow(img)

    plt.subplot(1, 4, 2)
    plt.title(f'{probs[r]:.3f}')
    plt.imshow(alphabet[r][1])

    plt.subplot(1, 4, 3)
    plt.title(f'{probs[rs[1]]:.3f}')
    plt.imshow(alphabet[rs[1]][1])

    plt.subplot(1, 4, 4)
    plt.title(f'{probs[rs[2]]:.3f}')
    plt.imshow(alphabet[rs[2]][1])
